In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from wordcloud import WordCloud
from pandas import DataFrame
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [25]:
file = pd.read_csv('/content/drive/My Drive/IST 736 Project/Final Table.csv')



#file = pd.read_csv('/content/drive/MyDrive/IST 736 Project/Twitter User Search.csv')

#file_b = pd.read_csv('/content/drive/MyDrive/IST 736 Project/Twitter Topic.csv')

In [26]:
np.unique(file['Party'])

file_b = file[file['Party'] == 'No-Detail']
file = file[file['Party'] != 'No-Detail']

In [27]:
filea = file.drop('Unnamed: 0', axis = 1)
filea = filea.drop('Label', axis = 1)
d_cv = filea[filea['Vectorizer'] == "CV (Not Binary)"]
d_cv2 = filea[filea['Vectorizer'] == "CV (Binary)"]
d_tf = filea[filea['Vectorizer'] == "TF_IDF"]
np.unique(filea.Vectorizer)

array(['CV (Binary)', 'CV (Not Binary)', 'TF_IDF'], dtype=object)

In [28]:
d_cv = d_cv.drop('Vectorizer', axis = 1)
d_cv2 = d_cv2.drop('Vectorizer', axis = 1)
d_tf = d_tf.drop('Vectorizer', axis = 1)

In [29]:
def model_run(dataset_input, model_input, model_label, n_fold_cv_input, vect_label):
    Train_PCT_Sent = []
    Test_PCT_Sent = []
    
    for i in range(0,n_fold_cv_input):
        ## Create the testing set - grab a sample from the training set. 
        ## Be careful. Notice that right now, our train set is sorted by label.
        ## If your train set is large enough, you can take a random sample.
        from sklearn.model_selection import train_test_split

        TrainDF, TestDF = train_test_split(dataset_input, test_size=0.4)
        #TrainDFD, TestDFB = train_test_split(d_cv_desc, test_size=0.3)


        ##-----------------------------------------------------------------
        ##
        ## Now we have a training set and a testing set. 
        #print("The training set is:")
        #print(TrainDF)
        #print("The testing set is:")
        #print(TestDF)

        ## IMPORTANT - !!!!!!! SEPERATE LABELS !!!!!!
        ## Save labels
        TestLabels=TestDF['Party']
        #print(TestLabels)
        ## remove labels
        TestDF = TestDF.drop(["Party"], axis=1)
        #print(TestDF)
        ## When you look up this model, you learn that it wants the 
        ## DF seperate from the labels
        TrainDF_nolabels=TrainDF.drop(["Party"], axis=1)
        #print(TrainDF_nolabels)
        TrainLabels=TrainDF["Party"]
        #print(TrainLabels)

        MyModelNB= model_input

        MyModelNB.fit(TrainDF_nolabels, TrainLabels)

        Prediction = MyModelNB.predict(TrainDF_nolabels)
        #print("The prediction from NB is:")
        #print(Prediction)
        #print("The actual labels are:")
        #print(TestLabels)
        ## confusion matrix
        from sklearn.metrics import confusion_matrix
        ## The confusion matrix is square and is labels X labels
        ## We ahve two labels, so ours will be 2X2
        #The matrix shows
        ## rows are the true labels
        ## columns are predicted
        ## it is al[habetical
        ## The numbers are how many 
        cnf_matrix_train = confusion_matrix(TrainLabels, Prediction)
        print("The confusion matrix for training is:")
        print(cnf_matrix_train, '\n')
        ### prediction probabilities
        ## columns are the labels in alphabetical order
        ## The decinal in the matrix are the prob of being
        ## that label
        #print(np.round(MyModelNB.predict_proba(TestDF),4))

        pct_correct = (100*((cnf_matrix_train[0,0]+cnf_matrix_train[1,1])/
                 (cnf_matrix_train[0,0] + cnf_matrix_train[0,1] + cnf_matrix_train[1,0] + cnf_matrix_train[1,1]))).round(3)



        print("Percent Correct for Train Data: ", pct_correct)

        Train_PCT_Sent.append(pct_correct)


        Prediction = MyModelNB.predict(TestDF)
        #print("The prediction from NB is:")
        #print(Prediction)
        #print("The actual labels are:")
        #print(TestLabels)
        ## confusion matrix
        from sklearn.metrics import confusion_matrix
        ## The confusion matrix is square and is labels X labels
        ## We ahve two labels, so ours will be 2X2
        #The matrix shows
        ## rows are the true labels
        ## columns are predicted
        ## it is al[habetical
        ## The numbers are how many 
        cnf_matrix_test = confusion_matrix(TestLabels, Prediction)
        print("The confusion matrix for testing is:")
        print(cnf_matrix_test, '\n')
        ### prediction probabilities
        ## columns are the labels in alphabetical order
        ## The decinal in the matrix are the prob of being
        ## that label
        #print(np.round(MyModelNB.predict_proba(TestDF),4))

        pct_correct = (100*((cnf_matrix_test[0,0]+cnf_matrix_test[1,1])/
                 (cnf_matrix_test[0,0] + cnf_matrix_test[0,1] + cnf_matrix_test[1,0] + cnf_matrix_test[1,1]))).round(3)


        print("Percent Correct for Test Data: ", pct_correct)

        Test_PCT_Sent.append(pct_correct)   
        
    Comparison = DataFrame(list(zip(Train_PCT_Sent,Test_PCT_Sent)), columns = ['Train', 'Test'])

    #print(n_fold_cv_input, ' Fold CV Average Train Data Classification for Sentiment: ', round(Comparison['Train'].mean(),2), "Percent")
    #print(n_fold_cv_input, ' Fold CV Average Test Data Classification: for Sentiment:', round(Comparison['Test'].mean(),2), "Percent")

    Comparison1 = Comparison
    
    Comparison = Comparison

    Comparison.loc['Mean'] = Comparison.mean()

    Comparison.reset_index(inplace = True)

    Comparison.columns = ['Model', 'Train', 'Test']
    
    #Comparison.columns = ['Model', 'Train', 'Test']

    Comparison_mean = (Comparison[Comparison['Model']== "Mean"]).round(2)

    Comparison_mean['Model'] = model_label

    Comparison_mean['Vectorizer'] = vect_label

    Comparison_mean = Comparison_mean[['Model', 'Vectorizer', 'Train', 'Test']]
    
    Comparison1.loc['Mean'] = Comparison1.mean()
    
    Comparison1['Model'] = model_label

    Comparison1['Vectorizer'] = vect_label

    Comparison1 = Comparison1[['Model', 'Vectorizer', 'Train', 'Test']]

    Comparison1.reset_index(inplace = True)

    return(Comparison_mean)

In [30]:
n_fold_validation = 1

In [31]:


model_run(d_cv2, BernoulliNB(),"BernoulliNB", n_fold_validation, "CountVectorizer (Bindary)")


The confusion matrix for training is:
[[1172  165]
 [  22 1341]] 

Percent Correct for Train Data:  93.074
The confusion matrix for testing is:
[[670 243]
 [107 780]] 

Percent Correct for Test Data:  80.556


,Model,Vectorizer,Train,Test
1,BernoulliNB,CountVectorizer (Bindary),93.07,80.56


In [32]:
model_run(d_cv, MultinomialNB(), "Multinomial NB", n_fold_validation, "CountVectorizer")


The confusion matrix for training is:
[[1333   18]
 [  99 1250]] 

Percent Correct for Train Data:  95.667
The confusion matrix for testing is:
[[794 105]
 [219 682]] 

Percent Correct for Test Data:  82.0


,Model,Vectorizer,Train,Test
1,Multinomial NB,CountVectorizer,95.67,82.0


In [33]:
model_run(d_tf, MultinomialNB(), "Multinomial NB", n_fold_validation, "TF-IDF")


The confusion matrix for training is:
[[1369    9]
 [ 178 1144]] 

Percent Correct for Train Data:  93.074
The confusion matrix for testing is:
[[801  71]
 [374 554]] 

Percent Correct for Test Data:  75.278


,Model,Vectorizer,Train,Test
1,Multinomial NB,TF-IDF,93.07,75.28


In [34]:
model_run(d_cv2, MultinomialNB(), "Multinomial NB (Binary)", n_fold_validation, "CountVectorizer (Binary)")


The confusion matrix for training is:
[[1299   44]
 [  74 1283]] 

Percent Correct for Train Data:  95.63
The confusion matrix for testing is:
[[793 114]
 [199 694]] 

Percent Correct for Test Data:  82.611


,Model,Vectorizer,Train,Test
1,Multinomial NB (Binary),CountVectorizer (Binary),95.63,82.61


In [35]:
model_run(d_tf, SVC(C=1, kernel = "linear", gamma = 'auto'), "SVC (Linear)", n_fold_validation, "TF-IDF")


The confusion matrix for training is:
[[1274   65]
 [  46 1315]] 

Percent Correct for Train Data:  95.889
The confusion matrix for testing is:
[[665 246]
 [190 699]] 

Percent Correct for Test Data:  75.778


,Model,Vectorizer,Train,Test
1,SVC (Linear),TF-IDF,95.89,75.78


In [36]:
model_run(d_tf, SVC(C=1, kernel = "poly", gamma = 'auto'), "SVC (Poly)", n_fold_validation, "TF-IDF")


The confusion matrix for training is:
[[   0 1346]
 [   0 1354]] 

Percent Correct for Train Data:  50.148
The confusion matrix for testing is:
[[  0 904]
 [  0 896]] 

Percent Correct for Test Data:  49.778


,Model,Vectorizer,Train,Test
1,SVC (Poly),TF-IDF,50.15,49.78


In [37]:
model_run(d_tf, SVC(C=1, kernel = "rbf", gamma = 'auto'), "SVC (Radial)", n_fold_validation, "TF-IDF")


The confusion matrix for training is:
[[   0 1333]
 [   0 1367]] 

Percent Correct for Train Data:  50.63
The confusion matrix for testing is:
[[  0 917]
 [  0 883]] 

Percent Correct for Test Data:  49.056


,Model,Vectorizer,Train,Test
1,SVC (Radial),TF-IDF,50.63,49.06


In [ ]:
model_run(d_cv, SVC(C=1, kernel = "linear", gamma = 'auto'), "SVC (Linear)", n_fold_validation, "CountVectorizer")


The confusion matrix for training is:
[[1339   23]
 [   1 1337]] 

Percent Correct for Train Data:  99.111
The confusion matrix for testing is:
[[654 234]
 [165 747]] 

Percent Correct for Test Data:  77.833


,Model,Vectorizer,Train,Test
1,SVC (Linear),CountVectorizer,99.11,77.83


In [ ]:
model_run(d_cv, SVC(C=1, kernel = "poly", gamma = 'auto'), "SVC (Poly)", n_fold_validation, "CountVectorizer")


The confusion matrix for training is:
[[   0 1346]
 [   0 1354]] 

Percent Correct for Train Data:  50.148
The confusion matrix for testing is:
[[  0 904]
 [  0 896]] 

Percent Correct for Test Data:  49.778


,Model,Vectorizer,Train,Test
1,SVC (Poly),CountVectorizer,50.15,49.78


In [ ]:
model_run(d_cv, SVC(C=1, kernel = "rbf", gamma = 'auto'), "SVC (Radial)", n_fold_validation, "CountVectorizer")

The confusion matrix for training is:
[[   0 1343]
 [   0 1357]] 

Percent Correct for Train Data:  50.259
The confusion matrix for testing is:
[[  0 907]
 [  0 893]] 

Percent Correct for Test Data:  49.611


,Model,Vectorizer,Train,Test
1,SVC (Radial),CountVectorizer,50.26,49.61


In [ ]:
model_run(d_cv, DecisionTreeClassifier(min_samples_leaf = 5, max_depth = 50), "Decision Tree", n_fold_validation, "CountVectorizer")

The confusion matrix for training is:
[[ 937  399]
 [ 168 1196]] 

Percent Correct for Train Data:  79.0
The confusion matrix for testing is:
[[493 421]
 [171 715]] 

Percent Correct for Test Data:  67.111


,Model,Vectorizer,Train,Test
1,Decision Tree,CountVectorizer,79.0,67.11


In [ ]:
model_run(d_cv, RandomForestClassifier(n_estimators = 500, n_jobs= -1), "RF", n_fold_validation, "CountVectorizer")

The confusion matrix for training is:
[[1314   20]
 [   0 1366]] 

Percent Correct for Train Data:  99.259
The confusion matrix for testing is:
[[710 206]
 [179 705]] 

Percent Correct for Test Data:  78.611


,Model,Vectorizer,Train,Test
1,RF,CountVectorizer,99.26,78.61


In [ ]:

def model_run_v2(dataset_input, model_input, model_label, n_fold_cv_input, vect_label):
    Train_PCT_Sent = []
    Test_PCT_Sent = []
    
    for i in range(0,n_fold_cv_input):
        ## Create the testing set - grab a sample from the training set. 
        ## Be careful. Notice that right now, our train set is sorted by label.
        ## If your train set is large enough, you can take a random sample.
        from sklearn.model_selection import train_test_split

        TrainDF, TestDF = train_test_split(dataset_input, test_size=0.4)
        #TrainDFD, TestDFB = train_test_split(d_cv_desc, test_size=0.3)


        ##-----------------------------------------------------------------
        ##
        ## Now we have a training set and a testing set. 
        #print("The training set is:")
        #print(TrainDF)
        #print("The testing set is:")
        #print(TestDF)

        ## IMPORTANT - !!!!!!! SEPERATE LABELS !!!!!!
        ## Save labels
        TestLabels=TestDF['Party']
        #print(TestLabels)
        ## remove labels
        TestDF = TestDF.drop(["Party"], axis=1)
        #print(TestDF)
        ## When you look up this model, you learn that it wants the 
        ## DF seperate from the labels
        TrainDF_nolabels=TrainDF.drop(["Party"], axis=1)
        #print(TrainDF_nolabels)
        TrainLabels=TrainDF["Party"]
        #print(TrainLabels)

        MyModelNB= model_input

        MyModelNB.fit(TrainDF_nolabels, TrainLabels)

        Prediction = MyModelNB.predict(TrainDF_nolabels)
        #print("The prediction from NB is:")
        #print(Prediction)
        #print("The actual labels are:")
        #print(TestLabels)
        ## confusion matrix
        from sklearn.metrics import confusion_matrix
        ## The confusion matrix is square and is labels X labels
        ## We ahve two labels, so ours will be 2X2
        #The matrix shows
        ## rows are the true labels
        ## columns are predicted
        ## it is al[habetical
        ## The numbers are how many 
        cnf_matrix_train = confusion_matrix(TrainLabels, Prediction)
        #print("The confusion matrix for training is:")
        #print(cnf_matrix_train, '\n')
        ### prediction probabilities
        ## columns are the labels in alphabetical order
        ## The decinal in the matrix are the prob of being
        ## that label
        #print(np.round(MyModelNB.predict_proba(TestDF),4))

        pct_correct = (100*((cnf_matrix_train[0,0]+cnf_matrix_train[1,1])/
                 (cnf_matrix_train[0,0] + cnf_matrix_train[0,1] + cnf_matrix_train[1,0] + cnf_matrix_train[1,1]))).round(3)



        #print("Percent Correct for Train Data: ", pct_correct)

        Train_PCT_Sent.append(pct_correct)


        Prediction = MyModelNB.predict(TestDF)
        #print("The prediction from NB is:")
        #print(Prediction)
        #print("The actual labels are:")
        #print(TestLabels)
        ## confusion matrix
        from sklearn.metrics import confusion_matrix
        ## The confusion matrix is square and is labels X labels
        ## We ahve two labels, so ours will be 2X2
        #The matrix shows
        ## rows are the true labels
        ## columns are predicted
        ## it is al[habetical
        ## The numbers are how many 
        cnf_matrix_test = confusion_matrix(TestLabels, Prediction)
        #print("The confusion matrix for testing is:")
        #print(cnf_matrix_test, '\n')
        ### prediction probabilities
        ## columns are the labels in alphabetical order
        ## The decinal in the matrix are the prob of being
        ## that label
        #print(np.round(MyModelNB.predict_proba(TestDF),4))

        pct_correct = (100*((cnf_matrix_test[0,0]+cnf_matrix_test[1,1])/
                 (cnf_matrix_test[0,0] + cnf_matrix_test[0,1] + cnf_matrix_test[1,0] + cnf_matrix_test[1,1]))).round(3)


        #print("Percent Correct for Test Data: "
        #      , pct_correct)

        Test_PCT_Sent.append(pct_correct)   
        
    Comparison = DataFrame(list(zip(Train_PCT_Sent,Test_PCT_Sent)), columns = ['Train', 'Test'])

    #print(n_fold_cv_input, ' Fold CV Average Train Data Classification for Sentiment: ', round(Comparison['Train'].mean(),2), "Percent")
    #print(n_fold_cv_input, ' Fold CV Average Test Data Classification: for Sentiment:', round(Comparison['Test'].mean(),2), "Percent")

    Comparison1 = Comparison
    
    Comparison = Comparison

    Comparison.loc['Mean'] = Comparison.mean()

    Comparison.reset_index(inplace = True)

    Comparison.columns = ['Model', 'Train', 'Test']
    
    #Comparison.columns = ['Model', 'Train', 'Test']

    Comparison_mean = (Comparison[Comparison['Model']== "Mean"]).round(2)

    Comparison_mean['Model'] = model_label

    Comparison_mean['Vectorizer'] = vect_label

    Comparison_mean = Comparison_mean[['Model', 'Vectorizer', 'Train', 'Test']]
    
    Comparison1.loc['Mean'] = Comparison1.mean()
    
    Comparison1['Model'] = model_label

    Comparison1['Vectorizer'] = vect_label

    Comparison1 = Comparison1[['Model', 'Vectorizer', 'Train', 'Test']]

    Comparison1.reset_index(inplace = True)

    return(Comparison_mean)





In [21]:
n_fold_validation = 5

d1 = model_run_v2(d_cv2, BernoulliNB(),"BernoulliNB", n_fold_validation, "CountVectorizer (Bindary)")
d2 = model_run_v2(d_cv, MultinomialNB(), "Multinomial NB", n_fold_validation, "CountVectorizer")
d3 = model_run_v2(d_tf, MultinomialNB(), "Multinomial NB", n_fold_validation, "TF-IDF")
d4 = model_run_v2(d_cv2, MultinomialNB(), "Multinomial NB (Binary)", n_fold_validation, "CountVectorizer (Binary)")
d5 = model_run_v2(d_tf, SVC(C=1, kernel = "linear", gamma = 'auto'), "SVC (Linear)", n_fold_validation, "TF-IDF")
#d6 = model_run_v2(d_tf, SVC(C=1, kernel = "poly", gamma = 'auto'), "SVC (Poly)", n_fold_validation, "TF-IDF")
#d7 = model_run_v2(d_tf, SVC(C=1, kernel = "rbf", gamma = 'auto'), "SVC (Radial)", n_fold_validation, "TF-IDF")
d8 = model_run_v2(d_cv, SVC(C=1, kernel = "linear", gamma = 'auto'), "SVC (Linear)", n_fold_validation, "CountVectorizer")
#d9 = model_run_v2(d_cv, SVC(C=1, kernel = "poly", gamma = 'auto'), "SVC (Poly)", n_fold_validation, "CountVectorizer")
#d10 = model_run_v2(d_cv, SVC(C=1, kernel = "rbf", gamma = 'auto'), "SVC (Radial)", n_fold_validation, "CountVectorizer")
d11 = model_run_v2(d_cv, DecisionTreeClassifier(min_samples_leaf = 5, max_depth = 50), "Decision Tree", n_fold_validation, "CountVectorizer")
d12 = model_run_v2(d_cv, RandomForestClassifier(n_estimators = 500, n_jobs= -1), "RF", n_fold_validation, "CountVectorizer")
final = pd.concat([d1,
                   d2,
                   d3,
                   d4, 
                   d5,
                   #d6,
                   #d7,
                   d8,
                   #d9, 
                   #d10
                   d11,
                   d12
                   ], join = "outer").reset_index()

final = final.drop('index', axis = 1)

final = final.sort_values('Test', ascending = False).reset_index()

final = final.drop('index', axis = 1)

final

,Model,Vectorizer,Train,Test
0,Multinomial NB,CountVectorizer,95.63,81.81
1,Multinomial NB (Binary),CountVectorizer (Binary),95.80,80.60
2,BernoulliNB,CountVectorizer (Bindary),93.40,80.59
3,SVC (Linear),CountVectorizer,99.00,78.42
4,RF,CountVectorizer,99.28,77.63
5,SVC (Linear),TF-IDF,95.90,76.42
6,Multinomial NB,TF-IDF,92.93,74.98
7,Decision Tree,CountVectorizer,78.45,66.97


In [22]:
final.to_excel('/content/drive/MyDrive/IST 736 Project/Compare Models.xlsx')

In [23]:
final.to_csv('/content/drive/MyDrive/IST 736 Project/Compare Models.csv')